In [9]:
using CUDA

In [10]:
sum = [1,2,3,4,3,2,1] |> cu

7-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 1
 2
 3
 4
 3
 2
 1

In [11]:
idx = zeros(6,size(sum,1)) |> cu

6×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [12]:
function testo!(idx,sum)
    i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

    if i <=size(sum,1)

        # Cleaning index matrices
        for m = 1:sum[i]
            idx[m,i] = 1
        end
    end
end

# Calculating Distance Matrix
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(idx,1),threads),
    testo!(idx,sum)
)

display(idx)

6×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [ ]:
        # First loop. Register all possible pairwise interactions based on
        # extensive sweep of the nearby cubes.
        for j = 1:size(points,1)
            d = euclidean(points,i,j) 
            # if d < r_max[i] && i!=j
            if d < 9.0 && i!=j
                idx_sum[i] += 1
                idx[idx_sum[i],i] = j
                dist[idx_sum[i],i] = d
            end

            # For an efficient implementation of the Gabriel method we sort the list
            # of possible interactions by distance
            if 1<j 
                for n=idx_sum[i]:-1:2
                    if dist[n,i]<dist[n-1,i]
                        idx[n,i],idx[n-1,i]   = idx[n-1,i],idx[n,i]
                        dist[n,i],dist[n-1,i] = dist[n-1,i],dist[n,i]
                    end
                end 
            end

        end

In [ ]:
# First loop. Register all possible pairwise interactions based on
# extensive sweep of the nearby cubes.
for j = 1:size(points,1)
    d = euclidean(points,i,j) 
    # if d < r_max[i] && i!=j
    if d < 5.0 && i!=j
        idx_sum[i] += 1
        idx[idx_sum[i],i] = j
        dist[idx_sum[i],i] = d
    end
end